In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# Folder where CSV files are stored
folder_path = './source_data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Household information

In [ ]:
# Data analysis on hhpub.xlsx file
file_path = os.path.join(folder_path, 'hhpub.csv')


df_original = pd.read_csv(file_path)
print("Shape:", df_original.shape)

display(df_original.head(5))


Shape: (129696, 58)


/tmp/ipython-input-2566174792.py:5: DtypeWarning: Columns (14,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(file_path)


,HOUSEID,TRAVDAY,SAMPSTRAT,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,PC,SPHONE,TAB,...,SMPLSRCE,WTHHFIN,HBHUR,HTHTNRNT,HTPPOPDN,HTRESDN,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN
0,30000007,2,3,1,3,5,7,2,1,2,...,2,187.314320,T,50,1500,750,750,20,750,300
1,30000008,5,2,1,2,4,8,1,1,2,...,2,69.513032,R,05,300,300,150,5,300,300
2,30000012,5,3,1,1,2,10,1,1,3,...,2,79.419586,C,80,17000,17000,5000,60,17000,7000
3,30000019,5,3,1,2,2,3,1,5,5,...,2,279.143588,S,40,300,300,150,50,750,300
4,30000029,3,3,1,2,2,5,2,5,1,...,2,103.240304,S,40,1500,750,750,40,1500,750


In [ ]:
# Create a working copy
df_working = df_original.copy()

print("Shape:", df_working.shape)

Shape: (129696, 58)


In [ ]:
# Keep only rows where HHSTATE is 'CA'
df_working = df_working[df_working['HHSTATE'] == 'CA']

# Count how many rows remain
row_count = len(df_working)
print(f"Number of rows where HHSTATE = 'CA': {row_count}")

# Preview first 5 rows
display(df_working.head(5))

Number of rows where HHSTATE = 'CA': 26099


,HOUSEID,TRAVDAY,SAMPSTRAT,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,PC,SPHONE,TAB,...,SMPLSRCE,WTHHFIN,HBHUR,HTHTNRNT,HTPPOPDN,HTRESDN,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN
6,30000041,4,3,1,2,2,11,1,1,1,...,2,788.614240,T,40,300,50,25,40,300,300
9,30000085,1,2,1,1,2,9,1,1,4,...,2,190.669041,U,50,17000,17000,5000,50,30000,17000
11,30000094,3,3,2,1,1,4,1,1,5,...,2,163.382292,T,20,300,300,150,20,300,300
19,30000155,1,1,1,1,2,-7,1,5,1,...,2,120.772451,C,30,3000,1500,1500,30,7000,1500
23,30000227,1,2,1,2,2,6,1,5,1,...,2,62.015790,S,05,3000,750,1500,5,750,300


In [ ]:
# Convert to string, strip spaces, remove leading single quote, pad with zero
df_working['HHFAMINC'] = df_working['HHFAMINC'].astype(str) \
                                .str.strip() \
                                .str.lstrip("'") \
                                .str.zfill(2)

# Check unique cleaned values
print("Unique cleaned HHFAMINC values:")
print(sorted(df_working['HHFAMINC'].unique()))


Unique cleaned HHFAMINC values:
['-7', '-8', '-9', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']


In [ ]:
# Define income code groups and descriptions
group_mapping = {
    1: {'codes': ['01','02','03','04'], 'desc': '$0 - 35,000'},
    2: {'codes': ['05','06'], 'desc': '$35,001 - 74,999'},
    3: {'codes': ['07','08'], 'desc': '$75,000 - 125,000'},
    4: {'codes': ['09','10','11'], 'desc': '$125,001 and above'}
}

# Function to assign group with formatted description
def assign_group_desc(code):
    code_str = str(code).zfill(2)  # ensure codes like 1 -> '01'
    for group_num, info in group_mapping.items():
        if code_str in info['codes']:
            return f"{group_num} ({info['desc']})"
    return 'Unknown'  # for missing or invalid codes

# Apply function to create a new column
df_working['HHFAMINC_GROUP_DESC'] = df_working['HHFAMINC'].apply(assign_group_desc)

# Check counts per group
group_counts = df_working['HHFAMINC_GROUP_DESC'].value_counts().sort_index()
print("Number of households per income group:")
display(group_counts)


Number of households per income group:


,count
HHFAMINC_GROUP_DESC,
"1 ($0 - 35,000)",6532
"2 ($35,001 - 74,999)",6860
"3 ($75,000 - 125,000)",6214
"4 ($125,001 and above)",5692
Unknown,801


In [ ]:
# First, clean the 'URBRUR' column in case it has quotes/spaces
df_working['URBRUR'] = df_working['URBRUR'].astype(str).str.strip().str.zfill(2)  # ensure '01' or '02'

# Map codes to readable labels
urb_rur_mapping = {
    '01': 'Urban',
    '02': 'Rural'
}
df_working['URBRUR_DESC'] = df_working['URBRUR'].map(urb_rur_mapping)

# Group by income group and urban/rural
grouped_counts = df_working.groupby(['HHFAMINC_GROUP_DESC', 'URBRUR_DESC']).size().unstack(fill_value=0)

print("Number of households per income group by Urban/Rural:")
display(grouped_counts)


Number of households per income group by Urban/Rural:


URBRUR_DESC,Rural,Urban
HHFAMINC_GROUP_DESC,,
"1 ($0 - 35,000)",865,5667
"2 ($35,001 - 74,999)",930,5930
"3 ($75,000 - 125,000)",774,5440
"4 ($125,001 and above)",554,5138
Unknown,88,713


In [ ]:
# Group by income group and Urban/Rural, calculate average vehicles per household
avg_vehicles = df_working.groupby(['HHFAMINC_GROUP_DESC', 'URBRUR_DESC'])['HHVEHCNT'].mean().unstack(fill_value=0)

print("Average number of vehicles per household by income group and Urban/Rural:")
display(avg_vehicles.round(2))  # round to 2 decimal places for readability


Average number of vehicles per household by income group and Urban/Rural:


URBRUR_DESC,Rural,Urban
HHFAMINC_GROUP_DESC,,
"1 ($0 - 35,000)",1.93,1.30
"2 ($35,001 - 74,999)",2.55,1.83
"3 ($75,000 - 125,000)",2.83,2.21
"4 ($125,001 and above)",3.25,2.43
Unknown,2.35,1.77


In [ ]:
#Rename household income colume to 'INC' and number of workers in household to 'W'
df_working = df_working.rename(columns={
    'HHFAMINC': 'INC',
    'WRKCOUNT': 'W'
})

In [ ]:
# Convert URBRUR column to numeric: Urban = 1, Rural = 0
df_working['URBRUR'] = df_working['URBRUR'].map({
    '01': 1,  # Urban
    '02': 0   # Rural
})

In [ ]:
# Column names

print(f'Number of Columns: {len(df_working.columns)}')
df_working.columns

Number of Columns: 60


Index(['HOUSEID', 'TRAVDAY', 'SAMPSTRAT', 'HOMEOWN', 'HHSIZE', 'HHVEHCNT',
       'INC', 'PC', 'SPHONE', 'TAB', 'WALK', 'BIKE', 'CAR', 'TAXI', 'BUS',
       'TRAIN', 'PARA', 'PRICE', 'PLACE', 'WALK2SAVE', 'BIKE2SAVE', 'PTRANS',
       'HHRELATD', 'DRVRCNT', 'CNTTDHH', 'HHSTATE', 'HHSTFIPS', 'NUMADLT',
       'YOUNGCHILD', 'W', 'TDAYDATE', 'HHRESP', 'LIF_CYC', 'MSACAT', 'MSASIZE',
       'RAIL', 'URBAN', 'URBANSIZE', 'URBRUR', 'SCRESP', 'CENSUS_D',
       'CENSUS_R', 'CDIVMSAR', 'HH_RACE', 'HH_HISP', 'HH_CBSA', 'RESP_CNT',
       'WEBUSE17', 'SMPLSRCE', 'WTHHFIN', 'HBHUR', 'HTHTNRNT', 'HTPPOPDN',
       'HTRESDN', 'HTEEMPDN', 'HBHTNRNT', 'HBPPOPDN', 'HBRESDN',
       'HHFAMINC_GROUP_DESC', 'URBRUR_DESC'],
      dtype='object')

All we need is some of the columns.  
Therefore filtering is needed.

In [ ]:
df_working_filtered = df_working.drop(columns=[ 'TRAVDAY', 'SAMPSTRAT', 'HOMEOWN', 'PC', 'SPHONE', 'TAB', 'WALK', 'BIKE', 'CAR', 'TAXI', 'BUS',
       'TRAIN', 'PARA', 'PRICE', 'PLACE', 'WALK2SAVE', 'BIKE2SAVE', 'PTRANS',
       'HHRELATD', 'DRVRCNT', 'CNTTDHH', 'NUMADLT',
       'YOUNGCHILD', 'TDAYDATE', 'HHRESP', 'LIF_CYC', 'MSACAT', 'MSASIZE',
       'RAIL', 'URBAN', 'URBANSIZE','SCRESP', 'CENSUS_D',
       'CENSUS_R', 'CDIVMSAR', 'HH_RACE', 'HH_HISP', 'HH_CBSA', 'RESP_CNT',
       'WEBUSE17', 'SMPLSRCE', 'WTHHFIN', 'HBHUR', 'HTHTNRNT', 'HTPPOPDN',
       'HTRESDN', 'HTEEMPDN', 'HBHTNRNT', 'HBPPOPDN', 'HBRESDN'], axis=1, inplace = False)
display(df_working_filtered.head(5))

,HOUSEID,HHSIZE,HHVEHCNT,INC,HHSTATE,HHSTFIPS,W,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC
6,30000041,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban
9,30000085,1,2,09,CA,6,1,1,"4 ($125,001 and above)",Urban
11,30000094,1,1,04,CA,6,1,1,"1 ($0 - 35,000)",Urban
19,30000155,1,2,-7,CA,6,1,1,Unknown,Urban
23,30000227,2,2,06,CA,6,0,1,"2 ($35,001 - 74,999)",Urban


## 2. Vehicle Information

In [ ]:
# Data analysis on vehicle information
# Path to the CSV file
file_path = os.path.join(folder_path, 'vehpub.csv')

# Read the CSV
df_vehpub = pd.read_csv(file_path)

In [ ]:
df_vehpub.columns

Index(['HOUSEID', 'VEHID', 'VEHYEAR', 'VEHAGE', 'MAKE', 'MODEL', 'FUELTYPE',
       'VEHTYPE', 'WHOMAIN', 'OD_READ', 'HFUEL', 'VEHOWNED', 'VEHOWNMO',
       'ANNMILES', 'HYBRID', 'PERSONID', 'TRAVDAY', 'HOMEOWN', 'HHSIZE',
       'HHVEHCNT', 'HHFAMINC', 'DRVRCNT', 'HHSTATE', 'HHSTFIPS', 'NUMADLT',
       'WRKCOUNT', 'TDAYDATE', 'LIF_CYC', 'MSACAT', 'MSASIZE', 'RAIL', 'URBAN',
       'URBANSIZE', 'URBRUR', 'CENSUS_D', 'CENSUS_R', 'CDIVMSAR', 'HH_RACE',
       'HH_HISP', 'HH_CBSA', 'SMPLSRCE', 'WTHHFIN', 'BESTMILE', 'BEST_FLG',
       'BEST_EDT', 'BEST_OUT', 'HBHUR', 'HTHTNRNT', 'HTPPOPDN', 'HTRESDN',
       'HTEEMPDN', 'HBHTNRNT', 'HBPPOPDN', 'HBRESDN', 'GSYRGAL', 'GSTOTCST',
       'FEGEMPG', 'FEGEMPGA', 'GSCOST', 'FEGEMPGF'],
      dtype='object')

In [ ]:
# Create a working copy
df_vehpub_working = df_vehpub.copy()

print("Shape:", df_vehpub_working.shape)

Shape: (256115, 60)


In [ ]:
# Keep only rows where HHSTATE is 'CA'
df_vehpub_working= df_vehpub_working[df_vehpub_working['HHSTATE'] == 'CA']
del df_vehpub_working['VEHYEAR']
# Count how many rows remain
row_count = len(df_vehpub_working)
print(f"Number of rows where HHSTATE = 'CA': {row_count}")

# Preview first 5 rows
display(df_vehpub_working.head(5))

Number of rows where HHSTATE = 'CA': 52215


,HOUSEID,VEHID,VEHAGE,MAKE,MODEL,FUELTYPE,VEHTYPE,WHOMAIN,OD_READ,HFUEL,...,HTEEMPDN,HBHTNRNT,HBPPOPDN,HBRESDN,GSYRGAL,GSTOTCST,FEGEMPG,FEGEMPGA,GSCOST,FEGEMPGF
17,30000041,1,1,49,49046,3,1,2,1532,4,...,25,40,300,300,331.251446,947.020280,50,-9,2.858917,1
18,30000041,2,6,49,49046,3,1,1,40560,4,...,25,40,300,300,207.269346,592.565787,48,-9,2.858917,1
27,30000085,1,13,49,49482,1,4,1,145000,-1,...,5000,50,30000,17000,902.362298,2579.778614,15,-9,2.858917,1
28,30000085,2,13,02,2407,1,3,1,175000,-1,...,5000,50,30000,17000,459.193033,1312.794615,23,-9,2.858917,1
31,30000094,1,8,37,37031,1,1,1,-88,-1,...,150,20,300,300,229.671414,656.611434,27,-9,2.858917,1


In [ ]:
# Remove rows where VEHFUEL is -8 or -7 (Invalid/Uncertain Input)
df_vehpub_working = df_vehpub_working[~df_vehpub_working['FUELTYPE'].isin([-8, -7])]

# Check the result
print("Shape after removing FUELTYPE = -8 and -7:", df_vehpub_working.shape)

# Verify unique values
print("Unique values in FUELTYPE after removal:")
print(sorted(df_vehpub_working['FUELTYPE'].unique()))

Shape after removing FUELTYPE = -8 and -7: (52153, 59)
Unique values in FUELTYPE after removal:
[np.int64(1), np.int64(2), np.int64(3), np.int64(97)]


In [ ]:
# Remove rows where ANNMILES == -9  (Invalid/Uncertain Input)
df_vehpub_working = df_vehpub_working[df_vehpub_working['ANNMILES'] != -9]

# Check the result
print("Shape after removing ANNMILES = -9:", df_vehpub_working.shape)

# Optional: check unique values in ANNMILES
print("Unique values in ANNMILES after removal:")
print(sorted(df_vehpub_working['ANNMILES'].unique()))

Shape after removing ANNMILES = -9: (38719, 59)
Unique values in ANNMILES after removal:
[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(10), np.int64(12), np.int64(14), np.int64(15), np.int64(16), np.int64(18), np.int64(20), np.int64(22), np.int64(24), np.int64(25), np.int64(26), np.int64(28), np.int64(29), np.int64(30), np.int64(32), np.int64(34), np.int64(35), np.int64(39), np.int64(40), np.int64(43), np.int64(44), np.int64(45), np.int64(48), np.int64(50), np.int64(54), np.int64(55), np.int64(60), np.int64(65), np.int64(66), np.int64(68), np.int64(70), np.int64(72), np.int64(75), np.int64(80), np.int64(84), np.int64(85), np.int64(87), np.int64(90), np.int64(92), np.int64(96), np.int64(100), np.int64(106), np.int64(109), np.int64(110), np.int64(113), np.int64(120), np.int64(121), np.int64(125), np.int64(129), np.int64(130), np.int64(133), np.int64(135), np.int64(140), np.int64(144), np.int64(145), np.int64

In [ ]:
# Convert to string, strip spaces, remove leading single quote, pad with zero
df_vehpub_working['HHFAMINC'] = df_vehpub_working['HHFAMINC'].astype(str) \
                                .str.strip() \
                                .str.lstrip("'") \
                                .str.zfill(2)

# Check unique cleaned values
print("Unique cleaned HHFAMINC values:")
print(sorted(df_vehpub_working['HHFAMINC'].unique()))

Unique cleaned HHFAMINC values:
['-7', '-8', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']


In [ ]:
# Remove rows where HHFAMINC is '-7' or '-8'
df_vehpub_working = df_vehpub_working[~df_vehpub_working['HHFAMINC'].isin(['-7', '-8'])]

# Check the shape after removal
print("Shape after removing HHFAMINC= -7 or -8:", df_vehpub_working.shape)

# Optional: check unique values remaining
print("Unique HHFAMINC values after removal:")
print(sorted(df_vehpub_working['HHFAMINC'].unique()))

Shape after removing HHFAMINC= -7 or -8: (37854, 59)
Unique HHFAMINC values after removal:
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']


In [ ]:
# Define income code groups and descriptions
group_mapping = {
    1: {'codes': ['01','02','03','04'], 'desc': '$0 - 35,000'},
    2: {'codes': ['05','06'], 'desc': '$35,001 - 74,999'},
    3: {'codes': ['07','08'], 'desc': '$75,000 - 125,000'},
    4: {'codes': ['09','10','11'], 'desc': '$125,001 and above'}
}

# Function to assign group with formatted description
def assign_group_desc(code):
    code_str = str(code).zfill(2)  # ensure codes like 1 -> '01'
    for group_num, info in group_mapping.items():
        if code_str in info['codes']:
            return f"{group_num} ({info['desc']})"
    return 'Unknown'  # for missing or invalid codes

# Apply function to create a new column
df_vehpub_working['HHFAMINC_GROUP_DESC'] = df_vehpub_working['HHFAMINC'].apply(assign_group_desc)

# Check counts per group
group_counts = df_vehpub_working['HHFAMINC_GROUP_DESC'].value_counts().sort_index()
print("Number of vehicles per income group:")
display(group_counts)


Number of vehicles per income group:


,count
HHFAMINC_GROUP_DESC,
"1 ($0 - 35,000)",6097
"2 ($35,001 - 74,999)",9689
"3 ($75,000 - 125,000)",10719
"4 ($125,001 and above)",11349


In [ ]:
# First, clean the 'URBRUR' column in case it has quotes/spaces
df_vehpub_working['URBRUR'] = df_vehpub_working['URBRUR'].astype(str).str.strip().str.zfill(2)  # ensure '01' or '02'

# Map codes to readable labels
urb_rur_mapping = {
    '01': 'Urban',
    '02': 'Rural'
}
df_vehpub_working['URBRUR_DESC'] = df_vehpub_working['URBRUR'].map(urb_rur_mapping)

# Group by income group and urban/rural
grouped_counts = df_vehpub_working.groupby(['HHFAMINC_GROUP_DESC', 'URBRUR_DESC']).size().unstack(fill_value=0)

print("Number of vehicles per income group by Urban/Rural:")
display(grouped_counts)


Number of vehicles per income group by Urban/Rural:


URBRUR_DESC,Rural,Urban
HHFAMINC_GROUP_DESC,,
"1 ($0 - 35,000)",1253,4844
"2 ($35,001 - 74,999)",1825,7864
"3 ($75,000 - 125,000)",1690,9029
"4 ($125,001 and above)",1437,9912


In [ ]:
# Ensure vehicle age column is numeric
df_vehpub_working['VEHAGE'] = pd.to_numeric(df_vehpub_working['VEHAGE'], errors='coerce')  # convert invalid to NaN

# Remove rows where VEHAGE is -7 or -8
df_vehpub_working = df_vehpub_working[(df_vehpub_working['VEHAGE'] != -7) &
                                      (df_vehpub_working['VEHAGE'] != -8)]

# Group by income group and Urban/Rural, calculate average vehicle age
avg_vehicle_age = df_vehpub_working.groupby(['HHFAMINC_GROUP_DESC', 'URBRUR_DESC'])['VEHAGE'].mean().unstack(fill_value=0)

print("Average vehicle age by income group and Urban/Rural:")
display(avg_vehicle_age.round(1))  # round to 1 decimal places


Average vehicle age by income group and Urban/Rural:


URBRUR_DESC,Rural,Urban
HHFAMINC_GROUP_DESC,,
"1 ($0 - 35,000)",17.0,13.8
"2 ($35,001 - 74,999)",14.8,12.1
"3 ($75,000 - 125,000)",12.8,10.9
"4 ($125,001 and above)",11.2,9.5


In [ ]:
# Ensure ANNMILES is numeric
df_vehpub_working['ANNMILES'] = pd.to_numeric(df_vehpub_working['ANNMILES'], errors='coerce')

# Group by income group and Urban/Rural, calculate average annual miles
avg_annual_miles = df_vehpub_working.groupby(['HHFAMINC_GROUP_DESC', 'URBRUR_DESC'])['ANNMILES'].mean().unstack(fill_value=0)

print("Average annual miles per vehicle by income group and Urban/Rural:")
display(avg_annual_miles.round(0))  # round to 0 decimal places


Average annual miles per vehicle by income group and Urban/Rural:


URBRUR_DESC,Rural,Urban
HHFAMINC_GROUP_DESC,,
"1 ($0 - 35,000)",6017.0,7628.0
"2 ($35,001 - 74,999)",7758.0,8141.0
"3 ($75,000 - 125,000)",8664.0,8947.0
"4 ($125,001 and above)",9848.0,9341.0


In [ ]:
# Ensure Fuel Economy column is numeric
df_vehpub_working['FEGEMPG'] = pd.to_numeric(df_vehpub_working['FEGEMPG'], errors='coerce')  # convert invalid to NaN

# Count total valid rows for MPG
total_mpg_rows = len(df_vehpub_working)
print(f"Total rows with valid FEGEMPG: {total_mpg_rows}")

# Group by income group and Urban/Rural, calculate average MPG
avg_mpg = df_vehpub_working.groupby(
    ['HHFAMINC_GROUP_DESC', 'URBRUR_DESC']
)['FEGEMPG'].mean().unstack(fill_value=0)

print("Average miles per gallon by income group and Urban/Rural:")
display(avg_mpg.round(1))  # round to 1 decimal places

Total rows with valid FEGEMPG: 37749
Average miles per gallon by income group and Urban/Rural:


URBRUR_DESC,Rural,Urban
HHFAMINC_GROUP_DESC,,
"1 ($0 - 35,000)",21.4,23.1
"2 ($35,001 - 74,999)",21.5,23.5
"3 ($75,000 - 125,000)",22.3,24.2
"4 ($125,001 and above)",23.4,25.4


In [ ]:
# Rename household income, fuel economy, and annual miles columns
df_vehpub_working = df_vehpub_working.rename(columns={
    'HHFAMINC': 'INC',    # Household income
    'FEGEMPG': 'FUEL_ECON',     # Fuel economy (miles per gallon)
    'ANNMILES': 'VMT',    # Annual vehicle miles traveled
})

In [ ]:
# Convert URBRUR column to numeric: Urban = 1, Rural = 0
df_vehpub_working['URBRUR'] = df_vehpub_working['URBRUR'].map({
    '01': 1,  # Urban
    '02': 0   # Rural
})

In [ ]:
display(df_vehpub_working.head(5))

,HOUSEID,VEHID,VEHAGE,MAKE,MODEL,FUELTYPE,VEHTYPE,WHOMAIN,OD_READ,HFUEL,...,HBPPOPDN,HBRESDN,GSYRGAL,GSTOTCST,FUEL_ECON,FEGEMPGA,GSCOST,FEGEMPGF,HHFAMINC_GROUP_DESC,URBRUR_DESC
17,30000041,1,1,49,49046,3,1,2,1532,4,...,300,300,331.251446,947.020280,50,-9,2.858917,1,"4 ($125,001 and above)",Urban
18,30000041,2,6,49,49046,3,1,1,40560,4,...,300,300,207.269346,592.565787,48,-9,2.858917,1,"4 ($125,001 and above)",Urban
27,30000085,1,13,49,49482,1,4,1,145000,-1,...,30000,17000,902.362298,2579.778614,15,-9,2.858917,1,"4 ($125,001 and above)",Urban
28,30000085,2,13,02,2407,1,3,1,175000,-1,...,30000,17000,459.193033,1312.794615,23,-9,2.858917,1,"4 ($125,001 and above)",Urban
52,30000227,2,13,07,7472,1,4,1,111710,-1,...,750,300,399.788118,1142.960912,16,-9,2.858917,1,"2 ($35,001 - 74,999)",Urban


## 3. Gas Price and Tax Rate

In [ ]:
#As of August 2017, according to EIA, retail gasoline price is $3.023/gallon. Diesel retail price is $3.067/gallon.
# Federal fuel tax is $0.184/gallon for gasoline, is $0.244/gallon for diesel.
# State fuel tax is $0.463/gallon for gasoline, and $0.530/gallon for diesel.

GasPrice = 3.023
DieselPrice = 3.067
FedGasTax = 0.184
FedDieselTax = 0.244
StateGasTax = 0.463
StateDieselTax = 0.530


## 4. VMT-based Tax Rate Strategy

In [ ]:
display(df_vehpub_working.head(5))
print(df_vehpub_working.columns)
#display(df_vehpub_working['VMT'].head(5))

,HOUSEID,VEHID,VEHAGE,MAKE,MODEL,FUELTYPE,VEHTYPE,WHOMAIN,OD_READ,HFUEL,...,HBPPOPDN,HBRESDN,GSYRGAL,GSTOTCST,FUEL_ECON,FEGEMPGA,GSCOST,FEGEMPGF,HHFAMINC_GROUP_DESC,URBRUR_DESC
17,30000041,1,1,49,49046,3,1,2,1532,4,...,300,300,331.251446,947.020280,50,-9,2.858917,1,"4 ($125,001 and above)",Urban
18,30000041,2,6,49,49046,3,1,1,40560,4,...,300,300,207.269346,592.565787,48,-9,2.858917,1,"4 ($125,001 and above)",Urban
27,30000085,1,13,49,49482,1,4,1,145000,-1,...,30000,17000,902.362298,2579.778614,15,-9,2.858917,1,"4 ($125,001 and above)",Urban
28,30000085,2,13,02,2407,1,3,1,175000,-1,...,30000,17000,459.193033,1312.794615,23,-9,2.858917,1,"4 ($125,001 and above)",Urban
52,30000227,2,13,07,7472,1,4,1,111710,-1,...,750,300,399.788118,1142.960912,16,-9,2.858917,1,"2 ($35,001 - 74,999)",Urban


Index(['HOUSEID', 'VEHID', 'VEHAGE', 'MAKE', 'MODEL', 'FUELTYPE', 'VEHTYPE',
       'WHOMAIN', 'OD_READ', 'HFUEL', 'VEHOWNED', 'VEHOWNMO', 'VMT', 'HYBRID',
       'PERSONID', 'TRAVDAY', 'HOMEOWN', 'HHSIZE', 'HHVEHCNT', 'INC',
       'DRVRCNT', 'HHSTATE', 'HHSTFIPS', 'NUMADLT', 'WRKCOUNT', 'TDAYDATE',
       'LIF_CYC', 'MSACAT', 'MSASIZE', 'RAIL', 'URBAN', 'URBANSIZE', 'URBRUR',
       'CENSUS_D', 'CENSUS_R', 'CDIVMSAR', 'HH_RACE', 'HH_HISP', 'HH_CBSA',
       'SMPLSRCE', 'WTHHFIN', 'BESTMILE', 'BEST_FLG', 'BEST_EDT', 'BEST_OUT',
       'HBHUR', 'HTHTNRNT', 'HTPPOPDN', 'HTRESDN', 'HTEEMPDN', 'HBHTNRNT',
       'HBPPOPDN', 'HBRESDN', 'GSYRGAL', 'GSTOTCST', 'FUEL_ECON', 'FEGEMPGA',
       'GSCOST', 'FEGEMPGF', 'HHFAMINC_GROUP_DESC', 'URBRUR_DESC'],
      dtype='object')


In [ ]:
df_vehpub_cleaned = df_vehpub_working.drop(columns=['VEHAGE', 'MAKE', 'MODEL', 'WHOMAIN',
                           'OD_READ', 'HFUEL', 'VEHOWNED', 'VEHOWNMO',
                           'HYBRID', 'PERSONID', 'TRAVDAY', 'HOMEOWN',
                           'HHSIZE', 'HHVEHCNT', 'DRVRCNT', 'NUMADLT',
                           'TDAYDATE', 'LIF_CYC', 'MSACAT', 'MSASIZE',
                           'RAIL', 'URBAN', 'URBANSIZE', 'CENSUS_D',
                           'CENSUS_R', 'CDIVMSAR', 'HH_RACE', 'HH_HISP',
                           'HH_CBSA', 'SMPLSRCE', 'WTHHFIN', 'BESTMILE',
                           'BEST_FLG', 'BEST_EDT', 'BEST_OUT', 'HBHUR',
                           'HTHTNRNT', 'HTPPOPDN', 'HTRESDN', 'HTEEMPDN',
                           'HBHTNRNT', 'HBPPOPDN', 'HBRESDN', 'GSYRGAL', 'GSTOTCST', 'FEGEMPGA', 'FEGEMPGF'], axis=1, inplace = False)
display(df_vehpub_cleaned.head(5))


,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,INC,HHSTATE,HHSTFIPS,WRKCOUNT,URBRUR,FUEL_ECON,GSCOST,HHFAMINC_GROUP_DESC,URBRUR_DESC
17,30000041,1,3,1,6000,11,CA,6,2,1,50,2.858917,"4 ($125,001 and above)",Urban
18,30000041,2,3,1,12000,11,CA,6,2,1,48,2.858917,"4 ($125,001 and above)",Urban
27,30000085,1,1,4,1500,09,CA,6,1,1,15,2.858917,"4 ($125,001 and above)",Urban
28,30000085,2,1,3,600,09,CA,6,1,1,23,2.858917,"4 ($125,001 and above)",Urban
52,30000227,2,1,4,400,06,CA,6,0,1,16,2.858917,"2 ($35,001 - 74,999)",Urban


**VEHTYPE**  
01=Automobile/Car/Station Wagon  
02=Van (Mini/Cargo/Passenger)   
03=SUV (Santa Fe, Tahoe, Jeep, etc.)   
04=Pickup Truck   
05=Other Truck   
06=RV (Recreational Vehicle)   
07=Motorcycle/Motorbike  

**Strategy 1: Based on Fuel Economy of Different Vehicle & Fuel Types**  
Diesel-fueled vehicles only account for 2.9% of all the vehicles, whereas 94.4% for gas-fueled ones. Thus we implement a uniform price policy for vehicles with different fuel types.  

Under this strategy, per-mile charges are inversely proportional to fuel efficiency, resulting in higher VMT fees for less fuel-efficient vehicle types and lower fees for more efficient ones, while preserving revenue neutrality. This pricing strategy is designed to improve equity across vehicle classes and better reflect differences in fuel consumption per mile traveled.



In [ ]:
# Strategy 1 Tax Level
print("Unique values in 'VEHTYPE' column:")
print(df_vehpub_cleaned['VEHTYPE'].unique())

# Remove rows where VEHTYPE is 97, -8, or -7
df_vehpub_cleaned = df_vehpub_cleaned[~df_vehpub_cleaned['VEHTYPE'].isin([97, -8, -7])]

# Calculate the mean FUEL_ECON for each VEHTYPE
df_Strategy1 = df_vehpub_cleaned.groupby('VEHTYPE', as_index=False)['FUEL_ECON'].mean()
df_Strategy1.rename(columns={'FUEL_ECON': 'MEAN_FUEL_ECON'}, inplace=True)

# Calculate the VMT fee rates using the mean FUEL_ECON
df_Strategy1['GASTAX_RATE'] = (StateGasTax) / df_Strategy1['MEAN_FUEL_ECON']
# Display the results

display(df_Strategy1)

Unique values in 'VEHTYPE' column:
[ 1  4  3  7  2  6  5 97 -8]


,VEHTYPE,MEAN_FUEL_ECON,GASTAX_RATE
0,1,26.580335,0.017419
1,2,18.855407,0.024555
2,3,19.968726,0.023186
3,4,15.956972,0.029016
4,5,16.258741,0.028477
5,6,14.000000,0.033071
6,7,50.000000,0.009260


In [ ]:
df_vehpub_to_add = df_vehpub_cleaned.drop(columns=['INC', 'HHSTATE','HHSTFIPS', 'WRKCOUNT', 'URBRUR', 'HHFAMINC_GROUP_DESC', 'URBRUR_DESC'], axis = 1)

df_merged = pd.merge(
    df_vehpub_to_add,
    df_working_filtered,
    on='HOUSEID',
    how='left'
)

display(df_merged)

,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,FUEL_ECON,GSCOST,HHSIZE,HHVEHCNT,INC,HHSTATE,HHSTFIPS,W,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC
0,30000041,1,3,1,6000,50,2.858917,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban
1,30000041,2,3,1,12000,48,2.858917,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban
2,30000085,1,1,4,1500,15,2.858917,1,2,09,CA,6,1,1,"4 ($125,001 and above)",Urban
3,30000085,2,1,3,600,23,2.858917,1,2,09,CA,6,1,1,"4 ($125,001 and above)",Urban
4,30000227,2,1,4,400,16,2.858917,2,2,06,CA,6,0,1,"2 ($35,001 - 74,999)",Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37667,40794179,1,2,4,18000,12,2.787083,1,1,06,CA,6,0,1,"2 ($35,001 - 74,999)",Urban
37668,40794233,1,1,4,8000,16,2.858917,2,3,08,CA,6,2,1,"3 ($75,000 - 125,000)",Urban
37669,40794233,2,1,1,10000,23,2.858917,2,3,08,CA,6,2,1,"3 ($75,000 - 125,000)",Urban
37670,40794233,3,1,1,5000,13,2.858917,2,3,08,CA,6,2,1,"3 ($75,000 - 125,000)",Urban


In [ ]:
# merged dataframes contains VMT-fee & CPM for each vehicle under strategy 1.
df_merged_VMT_Fee = pd.merge(
    df_merged,
    df_Strategy1[['VEHTYPE','GASTAX_RATE']],
    on='VEHTYPE',
    how='left'
)
df_merged_VMT_Fee['CPM'] = (GasPrice + FedGasTax) / df_merged_VMT_Fee['FUEL_ECON'] + df_merged_VMT_Fee['GASTAX_RATE']

display(df_merged_VMT_Fee.head())


,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,FUEL_ECON,GSCOST,HHSIZE,HHVEHCNT,INC,HHSTATE,HHSTFIPS,W,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC,GASTAX_RATE,CPM
0,30000041,1,3,1,6000,50,2.858917,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban,0.017419,0.081559
1,30000041,2,3,1,12000,48,2.858917,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban,0.017419,0.084231
2,30000085,1,1,4,1500,15,2.858917,1,2,09,CA,6,1,1,"4 ($125,001 and above)",Urban,0.029016,0.242816
3,30000085,2,1,3,600,23,2.858917,1,2,09,CA,6,1,1,"4 ($125,001 and above)",Urban,0.023186,0.162621
4,30000227,2,1,4,400,16,2.858917,2,2,06,CA,6,0,1,"2 ($35,001 - 74,999)",Urban,0.029016,0.229453


In [ ]:
file_path2 = os.path.join(folder_path, 'Strategy1_CPM.csv')
file_path3 = os.path.join(folder_path, 'Strategy2_CPM.csv')
df_Strategy1 = pd.read_csv(file_path2)
df_Strategy2 = pd.read_csv(file_path3)
df_Strategy1.rename(columns={'GASTAX_RATE': 'VMT_RATE'}, inplace=True)
df_Strategy1.rename(columns={'CPM': 'CPM_new'}, inplace=True)
df_Strategy1['CPM_old'] = df_Strategy2['CPM_old']
df_Strategy1['Delta_CPM'] = df_Strategy1['CPM_new'] - df_Strategy1['CPM_old']
df_Strategy1['VMT_Fee'] = df_Strategy1['VMT'] * df_Strategy1['VMT_RATE']
display(df_Strategy1.head())

,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,FUEL_ECON,GSCOST,HHSIZE,HHVEHCNT,INC,...,HHSTFIPS,W,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC,VMT_RATE,CPM_new,CPM_old,Delta_CPM,VMT_Fee
0,30000041,1,3,1,6000,50,2.858917,2,2,11,...,6,2,1,"4 ($125,001 and above)",Urban,0.017419,0.081559,0.073400,0.008159,104.513356
1,30000041,2,3,1,12000,48,2.858917,2,2,11,...,6,2,1,"4 ($125,001 and above)",Urban,0.017419,0.084231,0.076458,0.007773,209.026712
2,30000085,1,1,4,1500,15,2.858917,1,2,9,...,6,1,1,"4 ($125,001 and above)",Urban,0.029016,0.242816,0.244667,-0.001851,43.523294
3,30000085,2,1,3,600,23,2.858917,1,2,9,...,6,1,1,"4 ($125,001 and above)",Urban,0.023186,0.162621,0.159565,0.003056,13.911754
4,30000227,2,1,4,400,16,2.858917,2,2,6,...,6,0,1,"2 ($35,001 - 74,999)",Urban,0.029016,0.229453,0.229375,0.000078,11.606212


**Strategy 2: Different VMT fee rate for different income groups**  
In strategy 2 we propose an income-differentiated VMT fee structure to address equity considerations across different income groups. A uniform base VMT fee is first calculated as the weighted average of fuel tax payments per mile traveled across all vehicles, where total state gasoline tax revenue is divided by total vehicle miles traveled, accounting for vehicle-specific fuel economy.

The final VMT fee rate for each income group c is then determined by applying an equity adjustment factor (EAF) to the base fee. Using such a strategy, lower-income households are assigned adjustment factors below one, resulting in reduced per-mile charges, while higher-income households face slightly higher rates.


In [ ]:
# Strategy2
df_Strategy3 = df_Strategy1
base_rate1 = (((df_Strategy3['VMT'] / df_Strategy3['FUEL_ECON']).sum())*(StateGasTax))/(df_Strategy3['VMT'].sum())
print(f"base_rate1: {base_rate1}")

base_rate = 0.0215
groups = [
    "1 ($0 - 35,000)",
    "2 ($35,001 - 74,999)",
    "3 ($75,000 - 125,000)",
    "4 ($125,001 and above)"
]

VMT_rate = [
    base_rate * 0.9  ,  # group 1
    base_rate * 0.95,        # group 2
    base_rate * 1,              # group 3
    base_rate * 1.05         # group 4
]

VMT_Rate_S3 = pd.DataFrame({
    "HHFAMINC_GROUP_DESC": groups,
    "VMT_RATE": VMT_rate
})

display(VMT_Rate_S3)


,HHFAMINC_GROUP_DESC,VMT_RATE
0,"1 ($0 - 35,000)",0.019350
1,"2 ($35,001 - 74,999)",0.020425
2,"3 ($75,000 - 125,000)",0.021500
3,"4 ($125,001 and above)",0.022575


In [ ]:

df_Strategy3.drop(columns=['CPM_new', 'Delta_CPM', 'VMT_Fee', 'VMT_RATE'], inplace=True)
df_merged_VMT_Fee_S3 = pd.merge(
    df_Strategy3,
    VMT_Rate_S3,
    on='HHFAMINC_GROUP_DESC',
    how='left'
)

display(df_merged_VMT_Fee_S3.head())
display(df_merged_VMT_Fee_S3.columns)

base_rate1: 0.02159314756603981


,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,FUEL_ECON,GSCOST,HHSIZE,HHVEHCNT,INC,HHSTATE,HHSTFIPS,W,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC,CPM_old,TTCPM_VMT,TTCPM_FT,VMT_RATE
0,30000041,1,3,1,6000,50,2.858917,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban,0.073400,489.353356,440.40000,0.022575
1,30000041,2,3,1,12000,48,2.858917,2,2,11,CA,6,2,1,"4 ($125,001 and above)",Urban,0.076458,1010.776712,917.50000,0.022575
2,30000085,1,1,4,1500,15,2.858917,1,2,9,CA,6,1,1,"4 ($125,001 and above)",Urban,0.244667,364.223294,367.00000,0.022575
3,30000085,2,1,3,600,23,2.858917,1,2,9,CA,6,1,1,"4 ($125,001 and above)",Urban,0.159565,97.572623,95.73913,0.022575
4,30000227,2,1,4,400,16,2.858917,2,2,6,CA,6,0,1,"2 ($35,001 - 74,999)",Urban,0.229375,91.781212,91.75000,0.020425


Index(['HOUSEID', 'VEHID', 'FUELTYPE', 'VEHTYPE', 'VMT', 'FUEL_ECON', 'GSCOST',
       'HHSIZE', 'HHVEHCNT', 'INC', 'HHSTATE', 'HHSTFIPS', 'W', 'URBRUR',
       'HHFAMINC_GROUP_DESC', 'URBRUR_DESC', 'CPM_old', 'TTCPM_VMT',
       'TTCPM_FT', 'VMT_RATE'],
      dtype='object')

In [ ]:
df_merged_VMT_Fee_S3['CPM_new'] = (GasPrice + FedGasTax) / df_merged_VMT_Fee_S3['FUEL_ECON'] + df_merged_VMT_Fee_S3['VMT_RATE']
df_merged_VMT_Fee_S3['Delta_CPM'] = df_merged_VMT_Fee_S3['CPM_new'] - df_merged_VMT_Fee_S3['CPM_old']
df_merged_VMT_Fee_S3['VMT_Fee'] = df_merged_VMT_Fee_S3['VMT'] * df_merged_VMT_Fee_S3['VMT_RATE']
display(df_merged_VMT_Fee_S3.head())

vmt_fee_s3 = ((df_merged_VMT_Fee_S3['VMT_RATE'])*df_merged_VMT_Fee_S3['VMT']).sum()
fuel_tax = ((df_merged_VMT_Fee_S3['CPM_old'] - ((GasPrice + FedGasTax)/df_merged_VMT_Fee_S3['FUEL_ECON']))*df_merged_VMT_Fee_S3['VMT']).sum()
print(f"Total revenue of VMT fee based on Strategy0: {vmt_fee_s3}")
print(f"Total revenue of current fuel tax system: {fuel_tax}")

,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,FUEL_ECON,GSCOST,HHSIZE,HHVEHCNT,INC,...,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC,CPM_old,TTCPM_VMT,TTCPM_FT,VMT_RATE,CPM_new,Delta_CPM,VMT_Fee
0,30000041,1,3,1,6000,50,2.858917,2,2,11,...,1,"4 ($125,001 and above)",Urban,0.073400,489.353356,440.40000,0.022575,0.086715,0.013315,135.4500
1,30000041,2,3,1,12000,48,2.858917,2,2,11,...,1,"4 ($125,001 and above)",Urban,0.076458,1010.776712,917.50000,0.022575,0.089388,0.012929,270.9000
2,30000085,1,1,4,1500,15,2.858917,1,2,9,...,1,"4 ($125,001 and above)",Urban,0.244667,364.223294,367.00000,0.022575,0.236375,-0.008292,33.8625
3,30000085,2,1,3,600,23,2.858917,1,2,9,...,1,"4 ($125,001 and above)",Urban,0.159565,97.572623,95.73913,0.022575,0.162010,0.002445,13.5450
4,30000227,2,1,4,400,16,2.858917,2,2,6,...,1,"2 ($35,001 - 74,999)",Urban,0.229375,91.781212,91.75000,0.020425,0.220863,-0.008512,8.1700


TTCPM_VMT_S3: 55341596.007768765
TTCPM_FT_S3: 55436318.41194483
TTBase: 6963567.898499999
vmt_fee_s3: 6899014.768774999
fuel_tax: 6993737.172951071
TT_gas: 48442581.238993764


In [ ]:
# df_merged_VMT_Fee_S3.to_csv('/content/drive/MyDrive/MyFolder/CE-252/Strategy3_CPM_new.csv', index=False)

**Strategy 0: Baseline strategy**   
Using same logic in California Pilot Program 2017, adopting a uniform VMT fee rate.


In [ ]:
# Strategy 0
df_Strategy0 = df_merged_VMT_Fee_S3
GAS_RATE = StateGasTax/20
DIESEL_RATE = StateDieselTax/6.2
print(f"GAS_RATE: {GAS_RATE}")
print(f"DIESEL_RATE: {DIESEL_RATE}")

# df_Strategy4.loc[df_Strategy4['VEHTYPE'].isin([1,2,3,4,6,7]), 'VMT_RATE'] = GAS_RATE
# df_Strategy4.loc[df_Strategy4['VEHTYPE'].isin([5,97]) , 'VMT_RATE'] = DIESEL_RATE
df_Strategy0['VMT_RATE'] = GAS_RATE
df_Strategy0['CPM_new'] = (GasPrice + FedGasTax) / df_Strategy0['FUEL_ECON'] + df_Strategy0['VMT_RATE']
df_Strategy0['Delta_CPM'] = df_Strategy0['CPM_new'] - df_Strategy0['CPM_old']
#df_Strategy0.drop(columns=['VMT_Fee','TTCPM_VMT', 'TTCPM_FT'], inplace=True)
vmt_fee_s0 = ((df_Strategy0['VMT_RATE'])*df_Strategy0['VMT']).sum()
fuel_tax_s0 = ((df_Strategy0['CPM_old'] - ((GasPrice + FedGasTax)/df_Strategy0['FUEL_ECON']))*df_Strategy0['VMT']).sum()
print(f"Total revenue of VMT fee based on Strategy0: {vmt_fee_s0}")
print(f"Total revenue of current fuel tax system: {fuel_tax_s0}")
display(df_Strategy0.head())

GAS_RATE: 0.02315
DIESEL_RATE: 0.08548387096774193
vmt_fee_s4: 7497981.24885
fuel_tax_s4: 6993737.172951071


,HOUSEID,VEHID,FUELTYPE,VEHTYPE,VMT,FUEL_ECON,GSCOST,HHSIZE,HHVEHCNT,INC,...,URBRUR,HHFAMINC_GROUP_DESC,URBRUR_DESC,CPM_old,TTCPM_VMT,TTCPM_FT,VMT_RATE,CPM_new,Delta_CPM,VMT_Fee
0,30000041,1,3,1,6000,50,2.858917,2,2,11,...,1,"4 ($125,001 and above)",Urban,0.073400,520.29000,440.40000,0.02315,0.087290,0.013890,135.4500
1,30000041,2,3,1,12000,48,2.858917,2,2,11,...,1,"4 ($125,001 and above)",Urban,0.076458,1072.65000,917.50000,0.02315,0.089963,0.013504,270.9000
2,30000085,1,1,4,1500,15,2.858917,1,2,9,...,1,"4 ($125,001 and above)",Urban,0.244667,354.56250,367.00000,0.02315,0.236950,-0.007717,33.8625
3,30000085,2,1,3,600,23,2.858917,1,2,9,...,1,"4 ($125,001 and above)",Urban,0.159565,97.20587,95.73913,0.02315,0.162585,0.003020,13.5450
4,30000227,2,1,4,400,16,2.858917,2,2,6,...,1,"2 ($35,001 - 74,999)",Urban,0.229375,88.34500,91.75000,0.02315,0.223588,-0.005787,8.1700


In [ ]:
# df_Strategy4.to_csv('/content/drive/MyDrive/MyFolder/CE-252/Strategy0_CPM.csv', index = False)